pip install git+https://github.com/asimov-academy/Asimov_TV_connector.git

https://python.plainenglish.io/extracting-data-from-trading-view-253919ba7194

In [2]:
from tvDatafeed import TvDatafeed, Interval
from datetime import datetime, timedelta
import pandas as pd

username = 'jdonadelli'
password = open('passwd.txt', 'r').read()

tv = TvDatafeed(username, password)

error while signin
you are using nologin method, data you access may be limited


In [3]:
tv.search_symbol('PETR4','BMFBOVESPA')
df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 5000)
df.head(3)

Expecting value: line 1 column 1 (char 0)


,symbol,open,high,low,close,volume
datetime,,,,,,
2024-05-22 13:10:00,BMFBOVESPA:PETR4,35.866271,35.905171,35.837096,35.885721,203287.819038
2024-05-22 13:15:00,BMFBOVESPA:PETR4,35.875996,35.895446,35.866271,35.866271,93777.688904
2024-05-22 13:20:00,BMFBOVESPA:PETR4,35.866271,35.875996,35.846821,35.866271,24575.512772


In [ ]:
df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 10)
df

In [ ]:
url  = {'ibra50' :'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_IBr50.csv',
        'ibra100':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_IBr100.csv',
        'idiv'   :'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_Idiv.csv',
        'ibov'   :'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_Ibov.csv',
        'small'  :'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/Cart_Small.csv'
}

indice = pd.read_csv(url['small'] ,
                     sep=';' , decimal=',' , thousands='.' , header=1 , skiprows=1 , skipfooter=3 , encoding='latin-1',
                     index_col=False , engine='python')

indice.head()

In [ ]:
indice.tail()

In [ ]:
for idx,row in indice.iterrows():
    i=row['Código']
    if idx == 0:       
        df =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 800)
        df.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df['symbol']
    else:
        df1 =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 800)
        df1.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df1['symbol']

        df = pd.merge(df,df1,how='inner',left_index=True,right_index=True)

df.to_csv("data/tvohlc.csv" , sep=';' , decimal='.')

In [ ]:
def import_data(csv): 
    """ Renomeando as colunas para aritmetica com dataframe """
    def extrair_caracteres(s):
        import re
        pattern = re.compile(r"([^_]+)_")
    
        match = pattern.match(s)
        if match:
                return match.group(1)
        else:
             return None
    
    df = pd.read_csv(csv, sep=';' , decimal='.')
    df.index = pd.to_datetime(df['datetime'])#, format = "%y-%m-%d")
    df = df.drop('datetime', axis=1)
    
    close_columns    = []
    high_columns     = []
    low_columns      = []
    open_columns     = []
    volume_columns   = []

    for i in df.columns:
        if "close" in i:
            close_columns.append(i)
        elif "high" in i:
            high_columns.append(i)
        elif "low" in i:
            low_columns.append(i)
        elif "open_int" in i:
            open_int_columns.append(i)
        elif "open" in i:
            open_columns.append(i)
        elif "volume" in i:
            volume_columns.append(i)

    dicH = {s:extrair_caracteres(s) for s in high_columns}
    dicL = {s:extrair_caracteres(s) for s in low_columns}
    dicO = {s:extrair_caracteres(s) for s in open_columns}
    dicC = {s:extrair_caracteres(s) for s in close_columns}
    dicV = {s:extrair_caracteres(s) for s in volume_columns}
    
    close_df    = df[close_columns].rename(columns=dicC)
    high_df     = df[high_columns].rename(columns=dicH)
    low_df      = df[low_columns].rename(columns=dicL)
    open_df     = df[open_columns].rename(columns=dicO)
    volume_df   = df[volume_columns].rename(columns=dicV)
 
    return [close_df, high_df, low_df, open_df, volume_df]